In [162]:
def reorder2(table,df1_col,df2_col):
    df1=table.drop(table.columns[min(df2_col):],axis=1)
    df2=table.drop(table.columns[min(df1_col):min(df2_col)],axis=1)
    return (df1,df2)

def reorder3(table,df1_col,df2_col,df3_col):
    df1=table.drop(table.columns[min(df2_col):],axis=1)
    df2=table.drop(table.columns[min(df1_col):min(df2_col)].union(table.columns[min(df3_col):]),axis=1)
    df3=table.drop(table.columns[min(df1_col):min(df3_col)],axis=1)

    return (df1,df2,df3)

def verticalSplit_instrument(table_list, kw1='collar',kw2='swap'):

    '''
    Takes a list of tables and attempts to split them.
    Returns a new table_list that has the original tables split, if needed.
    '''
    split_dfs = []
    transformation_dict = {}
    #print(len(table_list))
    for table in table_list:
#         print('-'*10,'original table','-'*10)
#         display(HTML(table.to_html()))
        table = table.rename(str.lower, axis = "columns")
        for _ in range(3):
            table_columns = table.columns
            # we want the columns that have either the word 'oil' or 'bbl' in the column, but we want to omit columns that
            # have information purely about gas, ie, either 'gas' or 'btu'. We then add these two queries together, and remove
            # the columns that are duplicates between oil_cols and oil_cols2.
            # We then create a oil_cols, which we make sure does not have ANY gas information in it.
            collar_cols = [i for i,col in enumerate(table_columns) if (kw1 in col and kw2 not in col)]
            swap_cols = [i for i,col in enumerate(table_columns) if (kw2 in col and kw1 not in col)]
#             print(collar_cols,swap_cols)

            # if we have BOTH columns that ONLY HAVE oil information and gas information, we continue.
            if collar_cols!=[] and swap_cols!=[]:
                if min(collar_cols)<min(swap_cols): #first oil then gas and hten ngl 
                    df1,df2=reorder2(table,collar_cols,swap_cols)
                if min(collar_cols)>min(swap_cols):  
                    df1,df2=reorder2(table,swap_cols,collar_cols)
                split_dfs.append(df1)
                table=df2
                
            else: break
        split_dfs.append(table)
        
    return split_dfs


def verticalSplit_ZLU(table_list, verbose = False):

    '''
    Takes a list of tables and attempts to split them.
    Returns a new table_list that has the original tables split, if needed.
    '''
    split_dfs = []
    transformation_dict = {}
    verbose=True
    #print(len(table_list))
    for table in table_list:
#         print('-'*10,'original table','-'*10)
#         display(HTML(table.to_html()))
        table = table.rename(str.lower, axis = "columns")
        table_columns = table.columns
        # we want the columns that have either the word 'oil' or 'bbl' in the column, but we want to omit columns that
        # have information purely about gas, ie, either 'gas' or 'btu'. We then add these two queries together, and remove
        # the columns that are duplicates between oil_cols and oil_cols2.
        # We then create a oil_cols, which we make sure does not have ANY gas information in it.
        oil_cols = [i for i,col in enumerate(table_columns) if (("oil" in col or 'bbl' in col or 'barrel' in col) and (("ngl" not in col)  and ("gas" not in col) and ('btu' not in col)))]


        # this is the same process as above, but flipped.
        gas_cols = [i for i,col in enumerate(table_columns) if (("gas" in col or 'btu' in col) and (("ngl" not in col) and ("oil" not in col) and ('bbl' not in col)))]
        
        ngl_cols = [i for i,col in enumerate(table_columns) if ("ngl" in col and "oil" not in col)]
        # if we have BOTH columns that ONLY HAVE oil information and gas information, we continue.
        if min(len(oil_cols),1)+min(1,len(gas_cols))+min(1,len(ngl_cols))==3:
        
            if min(oil_cols)<min(gas_cols)<min(ngl_cols): #first oil then gas and hten ngl 
                df1,df2,df3=reorder3(table,oil_cols,gas_cols,ngl_cols)
            if min(oil_cols)<min(ngl_cols)<min(gas_cols):  
                df1,df2,df3=reorder3(table,oil_cols,ngl_cols,gas_cols)
            if min(gas_cols)<min(oil_cols)<min(ngl_cols):  
                df1,df2,df3=reorder3(table,gas_cols,oil_cols,ngl_cols)
            # add the newly split dfs to a new list, that we will eventually return.
            split_dfs.append(df1)
            split_dfs.append(df2)
            split_dfs.append(df3)
        elif oil_cols!=[] and gas_cols!=[]:
            if min(oil_cols)<min(gas_cols): #first oil then gas and hten ngl 
                df1,df2=reorder2(table,oil_cols,gas_cols)
            if min(oil_cols)>min(gas_cols):  
                df1,df2=reorder2(table,gas_cols,oil_cols)
            split_dfs.append(df1)
            split_dfs.append(df2)
        elif oil_cols!=[] and ngl_cols!=[]:
            if min(oil_cols)<min(ngl_cols): #first oil then gas and hten ngl 
                df1,df2=reorder2(table,oil_cols,ngl_cols)
            if min(oil_cols)>min(ngl_cols):  
                df1,df2=reorder2(table,ngl_cols,oil_cols)
            split_dfs.append(df1)
            split_dfs.append(df2)
        elif gas_cols!=[] and ngl_cols!=[]:
            if min(gas_cols)<min(ngl_cols): #first oil then gas and hten ngl 
                df1,df2=reorder2(table,gas_cols,ngl_cols)
            if min(gas_cols)>min(ngl_cols):  
                df1,df2=reorder2(table,ngl_cols,gas_cols)
            split_dfs.append(df1)
            split_dfs.append(df2)
        
        else:
            split_dfs.append(table)
    return split_dfs

In [6]:
# from collections import defaultdict, Counter
# def vertically_stacked_subtable_cleaner(dictionary, flagged_dict, debug = False):
#     '''
#     This function does two things:
#     1) Appends the splitted subtables to the list where the subtables are contained in
#     2) Deletes the original subtables

#     dictionary: dictionary with file_header as keys and list of dataframes as values
#     flagged_dict: output from get_vertically_stacked_subtables()
#     '''

#     debug_dict = defaultdict(list)

# ### Append subtables to original list under respective tickers
#     for ticker, indices in flagged_dict.items():
#         for index in indices:
#             df_original = dictionary[ticker][index]
#             # Create a temp version of df that removes parentheses and everything within it, for each cell

# #             print('\n')
#             # Assuming we don't have to split more than fifteen times per dataframe
#             for _ in range(15):
#                 for x_idx in range(df_original.shape[0]):
#                     df_temp = df_original.applymap(lambda x: re.sub(r'\([^()]*\)', "", x))
#                     # Find the first row index to split on, based of df_temp
#                     # The row must satisfy two conditions (1. no numbers within any cells, 2. the entine row not consist of only empty strings)
#                     if (not any(df_temp.iloc[x_idx].str.contains('\d'))) & (not all(df_temp.iloc[x_idx] == '')):
#                         break
#                 # If it's not a subtable anymore, break
#                 if (x_idx + 1) == df_original.shape[0]:
#                     break
#                 # We don't wanna split on the last row
#                 if x_idx == range(df_original.shape[0])[-1]:
#                     break
# #                 print('*******Original Dictionary*******')
# #                 print(ticker, index)
# #                 display(HTML(df_original.to_html()))
# #                 print(x_idx)
#                 df_split1 = df_original.iloc[:x_idx]
#                 df_split2 = df_original.iloc[x_idx:]
#                 # Keep the original columns as series for further usage
#                 columns_temp = df_split2.columns
#                 # Clear the columns to use set_col_header function
#                 df_split2.columns = ["" for x in columns_temp]
#                 if not any(df_split2.iloc[0,1:].str.contains('\d')):
#                     df_split2.columns=df_split2.iloc[0]
#                     df_split2=df_split2.iloc[1:]
#                 df_split2 = set_col_headers(df_split2)
#                 # Use original columns if empty OR if the new would_have_been column names are contained in columns_temp, or else, stick with the new ones
#                 df_split2.columns = [columns_temp[k] if (re.search('[\w]',v) == None or v in columns_temp[k]) else v for k,v in enumerate(df_split2.columns)]
#                 dictionary[ticker].append(df_split1)
# #                 print('*******Trimmed Dictionary*******')
# #                 display(HTML(df_split1.to_html()))
#                 debug_dict[ticker].append(df_split1)
#                 df_original = df_split2
#             # Grab the final piece of the dataframe
#             dictionary[ticker].append(df_original)
# #             display(HTML(df_split2.to_html()))
# #             print('\n')
# #             print('\n')
# #             print('*'*50)
#             debug_dict[ticker].append(df_split2)

# ### Delete the original subtables
#     if debug == False:
#         for k in flagged_dict.keys():
#             target_list = dictionary[k]
#             to_be_del = list(flagged_dict[k])
#             to_be_del.sort(reverse=True)
#             for index in to_be_del:
#                 del target_list[index]
#     else:
#         return debug_dict

In [7]:
def vertically_stacked_subtable_cleaner(dictionary, flagged_dict, exclusion=False, excluded_ticker=None):
    '''
    This function does two things:
    1) Appends the splitted subtables to the list where the subtables are contained in
    2) Deletes the original subtables

    dictionary: dictionary with file_header as keys and list of dataframes as values
    flagged_dict: output from get_vertically_stacked_subtables()
    exclusion: whether to exclude certain company within the flagged_dict or not
    excluded_ticker: the tickers to be excluded, as a typle of strings
    '''

    if exclusion == True:
        flagged_dict = {k: v for k, v in flagged_dict.items() if not k.startswith(excluded_ticker)}

### Append subtables to original list under respective tickers
    for ticker, indices in flagged_dict.items():
        for index in indices:
            df_original = dictionary[ticker][index]
            print(ticker,index)
            display(HTML(df_original.to_html()))
            # Assuming we don't have to split more than fifteen times per dataframe
            for _ in range(15):
                for x_idx in range(df_original.shape[0]):
                    # Create a temp version of df that removes parentheses and everything within it, for each cell
                    df_temp = df_original.applymap(lambda x: re.sub(r'\([^()]*\)', "", x))
                    # Find the first row index to split on, based of df_temp
                    # The row must satisfy two conditions (1. no numbers within any cells, 2. the entine row not consist of only empty strings)
                    if (not any(df_temp.iloc[x_idx].str.contains('\d'))) & (not all(df_temp.iloc[x_idx] == '')):
                        break
                # If it's not a subtable anymore, break
                if (x_idx + 1) == df_original.shape[0]:
                    break
                # We don't wanna split on the last row
                if x_idx == range(df_original.shape[0])[-1]:
                    break
                if df_original.shape[1]==1: break
                df_split1 = df_original.iloc[:x_idx]
                df_split2 = df_original.iloc[x_idx:]
                # Keep the original columns as series for further usage
                columns_temp = df_split2.columns
                # Clear the columns to use set_col_header function
                df_split2.columns = ["" for x in columns_temp]
                if not any(df_split2.iloc[0, 1:].str.contains('\d')):
                    df_split2.columns=df_split2.iloc[0]
                    df_split2=df_split2.iloc[1:]
                df_split2 = set_col_headers(df_split2)
                # Use original columns if empty OR if the new would_have_been column names are contained in columns_temp, or else, stick with the new ones
                df_split2.columns = [columns_temp[k] if (re.search('[\w]',v) == None or v in columns_temp[k]) else v for k,v in enumerate(df_split2.columns)]
                dictionary[ticker].append(df_split1)
                df_original = df_split2
            # Grab the final piece of the dataframe. If splitting not occured, we will append the original and have the duplicated be handled by drop_duplicates()
            dictionary[ticker].append(df_original)

### Delete the original subtables
    for k in flagged_dict.keys():
        target_list = dictionary[k]
        to_be_del = list(flagged_dict[k])
        to_be_del.sort(reverse=True)
        for index in to_be_del:
            del target_list[index]


In [187]:
%load_ext autoreload
%autoreload 2
import pickle
import re
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from zlu_functions import del_subset_col, set_col_headers, \
row_header_to_col,remove_rows_for_total_value,dfs_need_transpose, remove_cols_for_fair_value,collar_fill_empty,\
remove_empty_rows_and_cols,df_column_uniquify
from nel_functions import delete_uninformative,delete_empty,delete_duplicated, delete_subset, get_vertically_stacked_subtables #vertically_stacked_subtable_cleaner
from helper import volumeCleaner, dateCleaner, priceCleaner,trimDictonary,verticalSplit
pickle_in=open('../SEC_Data/Hedge/hedge_dfs_subset_8_18.pickle','rb')
df_dic=pickle.load(pickle_in)

delete_empty(df_dic)
df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
del_dfs_dic_0 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
del_dfs_dic_1 = delete_subset(df_dic)
df_dic={'APC20170217':df_dic['APC20170217']}

# keys=set(df_dic.keys())
# for k in keys:
#     if 'SWN' not in k:
#         del df_dic[k]


with open('../SEC_Data/8_18_outputs/output_before_spliting.csv','w') as f:
    pass
#preprocessing for all dfs
for k in df_dic.keys():
    #APC patch
    counter=0
    if 'APC' in k:
        for i in range(len(df_dic[k])):
            i=i-counter
            cond1='collar' not in "".join([x.lower() for x in df_dic[k][i].values.flatten()])
            cond2='swap' not in "".join([x.lower() for x in df_dic[k][i].values.flatten()])
            if cond1 and cond2: 
                counter+=1
                del df_dic[k][i]
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        df=set_col_headers(df) #set headers
        df=del_subset_col(df) #delete redundant cols
        df=remove_rows_for_total_value(df) #if any row for total value; save it
        df=remove_cols_for_fair_value(df)
        df_dic[k][i]=df
        with open('../SEC_Data/8_18_outputs/output_before_spliting.csv','a') as f:
            df=df_dic[k][i]
            f.writelines(k+'\n'+'table:'+str(i))
            try: df.to_csv(f,encoding='cp1252')
            except: df.to_csv(f,encoding='utf-8')

# #         print(k,'Original Table:',i,'-'*10)
# #         display(HTML(df.to_html()))




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [175]:
df=df_dic['APA20100226'][5]
[True if re.search('2\d{3}',col)!=None and re.search('\d2\d{3}',col)==None else re.search('\d',col)==None for col in df.iloc[0]]

[True, False, True]

In [188]:
for df in df_dic['APC20170217']:
    display(HTML(df.to_html()))

,",",",2017 Settlement",",2018 Settlement"
2,Oil,,
3,Three-Way Collars (MBbls/d),91,
4,Average price per barrel,,
5,Ceiling sold price (call),$59.80,
6,Floor purchased price (put),$50.00,
7,Floor sold price (put),$40.00,
8,Natural Gas,,
9,Three-Way Collars (thousand MMBtu/d),682,250
10,Average price per MMBtu,,
11,Ceiling sold price (call),$3.60,$3.54


In [147]:
#Vertical Split 
for k in df_dic.keys(): #apply Sam's spliting function
    if 'CLR' in k: continue #change 8/16/2019
    if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
    elif k=='BRY20110301':
        split_dfs = []
        for table in df_dic[k]:
            df1 = table.iloc[:, 0:3]
            df2 = table.iloc[:, 3:]
            split_dfs.append(df1)
            split_dfs.append(df2)
            df_dic[k]=split_dfs
    else: df_dic[k] = verticalSplit_ZLU(df_dic[k])
    df_dic[k]=verticalSplit_instrument(df_dic[k])
    if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
    else: df_dic[k] = verticalSplit_ZLU(df_dic[k])

In [148]:
for df in df_dic['CHK20100301']:
    display(HTML(df.to_html()))

,",,","volume,volume,(bbtu)","weighted average price,fixed,","weighted average price,put,(per mmbtu)","weighted average price,call,(per mmbtu)","weighted average price,differential,","cash flowhedge,cash flowhedge,","netpremiums,netpremiums,($ in millions)"
3,Natural Gas,,,,,,,
4,Swaps,,,,,,,
5,Q1|2010,63478,$7.59,,,,Yes,
6,Q2|2010,64781,7.27,,,,Yes,
7,Q3|2010,51972,7.32,,,,Yes,
8,Q4|2010,53212,7.40,,,,Yes,
9,2011,22210,7.99,,,,Yes,
11,Other Swaps(a),,,,,,,
12,Q1|2010,33890,7.22,,,,No,
13,Q2|2010,43680,7.61,,,,No,


,",,","volume,volume,(mbbls)","weighted average price,fixed,","weighted average price,put,(per bbl)","weighted average price,call,(per bbl)","weighted average price,differential,","cash flowhedge,cash flowhedge,","netpremiums,netpremiums,($ in millions)"
3,Oil,,,,,,,
4,Swaps,,,,,,,
5,Q1|2010,450,$85.86,,,,Yes,
6,Q2|2010,455,85.86,,,,Yes,
7,Q3|2010,460,85.86,,,,Yes,
8,Q4|2010,460,85.86,,,,Yes,
10,Other Swaps(c),,,,,,,
11,Q1|2010,360,91.96,,,,No,
12,Q2|2010,364,91.96,,,,No,
13,Q3|2010,368,91.96,,,,No,


,",,","december 31, 2009,volumehedged,","december 31, 2008,volumehedged,"
3,Natural gas (bbtu),,
4,Fixed-price swaps,492053,466800
5,Fixed-price collars,74240,457715
6,Fixed-price knockout swaps,38370,532660
7,Call options,996750,551555
8,Put options,Neg69620,Neg73000
9,Basis protection swaps,125469,219487
13,Oil (mbbl),,
14,Fixed-price swaps,5475,Neg310
15,Fixed-price collars,,730


In [159]:
for k in df_dic.keys():
    print(k)
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        if df.shape[1]==1: df=pd.DataFrame()
        elif isinstance(df, pd.DataFrame): #after new headers; ignore empty dfs
            df=del_subset_col(df) #delete redundant cols
            df=remove_rows_for_total_value(df) #if any row for total value; save it
            df=remove_cols_for_fair_value(df)
            df=row_header_to_col(df)
            df=dfs_need_transpose(df)
            df=collar_fill_empty(df)
            df=df_column_uniquify(df)
            df_dic[k][i]=remove_empty_rows_and_cols(df)


CLR20100226


In [160]:
df_dic['CLR20100226'][2]

,",,period and type of contract","volume inmbbls,volume inmbbls,volume inmbbls","swapsweightedaverage,swapsweightedaverage,swapsweightedaverage","collars,floors,range","collars,floors,weightedaverage","collars,ceilings,range","collars,ceilings,weightedaverage",additional
4,Swaps,905,$80.50,,,,,january 2010 june 2010
5,Collars,453,,$70.00,$70.00,$95.00,$95.00,january 2010 june 2010
7,Collars,644,,75.00,75.00,96.75,96.75,july 2010 december 2010
9,Collars,1278,,75.00,75.00,89.00,89.00,january 2011 december 2011


In [154]:
dfs_need_transpose(df_dic['CHK20100301'][2])

True True


,",,",natural gas (bbtu)Fixed-price swaps,natural gas (bbtu)Fixed-price collars,natural gas (bbtu)Fixed-price knockout swaps,natural gas (bbtu)Call options,natural gas (bbtu)Put options,natural gas (bbtu)Basis protection swaps,oil (mbbl)Fixed-price swaps,fixed-price collarsFixed-price knockout swaps,fixed-price cap-swapsCall options
1,"december 31, 2009,volumehedged,",492053,74240,38370,996750,Neg69620,125469,5475,6572,14975
2,"december 31, 2008,volumehedged,",466800,457715,532660,551555,Neg73000,219487,Neg310,12248,19355


In [141]:
for df in df_dic['SWN20080228']:
    display(HTML(df.to_html()))

,,volume,weighted average price to be swapped ($),weighted average floor price ($),weighted average ceiling price ($),weighted average basis differential ($),product_type,instrument_type
3,"2008 ,",55.7,8.27,-,-,-,natural gas (bcf),fixed price swaps
4,2009,56.0,8.20,-,-,-,natural gas (bcf),fixed price swaps
7,2008,48.0,-,7.92,11.60,-,natural gas (bcf),costless collars
8,2009,23.0,-,8.09,10.91,-,natural gas (bcf),costless collars
11,2008,63.2,-,-,-,Neg0.45,natural gas (bcf),basis swaps
12,2009,3.6,-,-,-,Neg0.44,natural gas (bcf),basis swaps
15,2008,8.0,-,-,-,Neg0.73,natural gas (bcf),matched-basis swaps


In [113]:
#Vertical Split
for k in df_dic.keys(): #apply Sam's spliting function
    print(k)
    if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
    elif "APC" in k:
        df_dic[k] = verticalSplit_instrument(df_dic[k],'two way','fixed price')
        df_dic[k] = verticalSplit_instrument(df_dic[k],'two way','fixed price')
        df_dic[k] = verticalSplit_instrument(df_dic[k],'two way','three way')
        df_dic[k] = verticalSplit_instrument(df_dic[k],'three way','fixed price')
        df_dic[k] = verticalSplit_instrument(df_dic[k],'basis swaps','fixed price')


    else: df_dic[k] = verticalSplit_ZLU(df_dic[k])
    df_dic[k]=verticalSplit_instrument(df_dic[k])
   
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        if df.shape[1]==1: df=pd.DataFrame()
        elif isinstance(df, pd.DataFrame): #after new headers; ignore empty dfs
            df=del_subset_col(df) #delete redundant cols
            df=remove_rows_for_total_value(df) #if any row for total value; save it
            df=remove_cols_for_fair_value(df)
            df=df_column_uniquify(df)
            df=remove_empty_rows_and_cols(df)
            
            row_lst=list(range(df.shape[0]))
            for ii in range(df.shape[0]):
                if not any(df.iloc[ii].str.contains('\d')): row_lst.remove(ii)
            df_dic[k][i]=df.iloc[row_lst]
            if sum(df_dic[k][i].shape)<=2: df_dic[k][i]=pd.DataFrame()

SWN20100225


In [114]:
for df in df_dic['SWN20100225']:
    display(HTML(df.to_html()))

,",,,,,natural gas (bcf),,fixed price swaps",",,,,volume,,,","weighted,average,price to be,swapped,($/mmbtu),,,","weighted,average,floor,price,($/mmbtu),,,","weighted,average,ceiling,price,($/mmbtu),,,","weighted,average,basis,differential,($/mmbtu),,,",instrument_type
8,2010,36.0,9.04,,,,
9,2011,30.0,6.69,,,,
12,2010,30.0,,6.80,8.43,,costless-collars
15,2010,46.5,,,,Neg0.37,basis swaps
16,2011,9.0,,,,Neg0.35,basis swaps


,"natural gas (bcf),,fixed price swaps",",,",instrument_type
3,2010,36.0,
4,2011,30.0,
7,2010,30.0,costless-collars


In [115]:
#Horizontally Split
del_dfs_dic3=get_vertically_stacked_subtables(df_dic)
vertically_stacked_subtable_cleaner(df_dic,del_dfs_dic3,exclusion=True, excluded_ticker=('MXC', 'OXY', 'HOC'))

delete_empty(df_dic)
df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
del_dfs_dic_2 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
del_dfs_dic_3 = delete_subset(df_dic)

for k in df_dic.keys(): #apply Sam's spliting function
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        if df.shape[1]==1: df=pd.DataFrame()
        elif isinstance(df, pd.DataFrame): #after new headers; ignore empty dfs
            df=del_subset_col(df) #delete redundant cols
            df_dic[k][i]=df_column_uniquify(df)
            df_dic[k][i]=remove_empty_rows_and_cols(df)

In [116]:
for df in df_dic['SWN20100225']:
    display(HTML(df.to_html()))

,",,,,,natural gas (bcf),,fixed price swaps",",,,,volume,,,","weighted,average,price to be,swapped,($/mmbtu),,,","weighted,average,floor,price,($/mmbtu),,,","weighted,average,ceiling,price,($/mmbtu),,,","weighted,average,basis,differential,($/mmbtu),,,",instrument_type
8,2010,36.0,9.04,,,,
9,2011,30.0,6.69,,,,
12,2010,30.0,,6.80,8.43,,costless-collars
15,2010,46.5,,,,Neg0.37,basis swaps
16,2011,9.0,,,,Neg0.35,basis swaps


In [96]:
with open('../SEC_Data/8_18_outputs/output_before_sam.csv','w') as f:
    pass
with open('../SEC_Data/8_18_outputs/output_after_sam.csv','w') as f:
    pass
for k in df_dic.keys():
    for i in range(len(df_dic[k])):
        with open('../SEC_Data/8_18_outputs/output_before_sam.csv','a') as f:
            df=df_dic[k][i]
            f.writelines(k+'\n'+'table:'+str(i))
            try: df.to_csv(f,encoding='cp1252')
            except: df.to_csv(f,encoding='utf-8')


for k in df_dic.keys():
    print(k)
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        if df.shape[0]>0: #apply Sam's functions to non-empty dfs
#             print(k+' ,table:'+str(i)+'-----After Preprocessing----')
#             display(HTML(df.to_html()))
            df = dateCleaner(df, key_words = "", stop_words = "")
            df = volumeCleaner(df, key_words = "", stop_words = "")
            df = priceCleaner(df, key_words = "", stop_words = "")
            print(k+' ,table:'+str(i)+'-----After Sam functions----')
            display(HTML(df.to_html()))
        else: df=pd.DataFrame()
        df_dic[k][i]=df

del_dfs_dic_5 = delete_uninformative(df_dic)
# print('1',len(df_dic))
delete_empty(df_dic)
# print('2',len(df_dic))
# for df in df_dic['SWN20040225']:
#     display(HTML(df.to_html()))
df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
# print('3',len(df_dic))
del_dfs_dic_2 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
# print('4',len(df_dic))
del_dfs_dic_3 = delete_subset(df_dic)
# print('5',len(df_dic))

for k in df_dic.keys():
    for i in range(len(df_dic[k])):
        with open('../SEC_Data/8_18_outputs/output_after_sam.csv','a') as f:
            df=df_dic[k][i]
            f.writelines(k+'\n'+'table:'+str(i))
            try: df.to_csv(f,encoding='cp1252')
            except: df.to_csv(f,encoding='utf-8')

SWN20100225
SWN20100225 ,table:0-----After Sam functions----


,vol_1,vol_1_type,vol_2,vol_2_type,vol_3,vol_3_type,vol_4,vol_4_type,vol_5,vol_5_type,vol_6,vol_6_type,instrument_type
8,2010,",,,,,natural gas (bcf),,fixed price swaps",36.0,",,,,volume,,,",9.04,"weighted,average,price to be,swapped,($/mmbtu),,,",,"weighted,average,floor,price,($/mmbtu),,,",,"weighted,average,ceiling,price,($/mmbtu),,,",,"weighted,average,basis,differential,($/mmbtu),,,",
9,2011,",,,,,natural gas (bcf),,fixed price swaps",30.0,",,,,volume,,,",6.69,"weighted,average,price to be,swapped,($/mmbtu),,,",,"weighted,average,floor,price,($/mmbtu),,,",,"weighted,average,ceiling,price,($/mmbtu),,,",,"weighted,average,basis,differential,($/mmbtu),,,",
12,2010,",,,,,natural gas (bcf),,fixed price swaps",30.0,",,,,volume,,,",,"weighted,average,price to be,swapped,($/mmbtu),,,",6.80,"weighted,average,floor,price,($/mmbtu),,,",8.43,"weighted,average,ceiling,price,($/mmbtu),,,",,"weighted,average,basis,differential,($/mmbtu),,,",costless-collars
15,2010,",,,,,natural gas (bcf),,fixed price swaps",46.5,",,,,volume,,,",,"weighted,average,price to be,swapped,($/mmbtu),,,",,"weighted,average,floor,price,($/mmbtu),,,",,"weighted,average,ceiling,price,($/mmbtu),,,",Neg0.37,"weighted,average,basis,differential,($/mmbtu),,,",basis swaps
16,2011,",,,,,natural gas (bcf),,fixed price swaps",9.0,",,,,volume,,,",,"weighted,average,price to be,swapped,($/mmbtu),,,",,"weighted,average,floor,price,($/mmbtu),,,",,"weighted,average,ceiling,price,($/mmbtu),,,",Neg0.35,"weighted,average,basis,differential,($/mmbtu),,,",basis swaps


In [97]:
df_dic.keys()

dict_keys([])

In [163]:
for k in df_dic.keys():
    for df in df_dic[k]:
        display(HTML(df.to_html()))

,"period and type of contract,period and type of contract,period and type of contract","volumeinmbbls,volumeinmbbls,volumeinmbbls","swapsweightedaverage,swapsweightedaverage,swapsweightedaverage","collars,floors,range","collars,floors,weightedaverage","collars,ceilings,range","collars,ceilings,weightedaverage",additional
4,Swaps,343,$84.00,,,,,january 2010 march 2010
5,Collars,148,,75.00,75.00,96.40,96.40,january 2010 march 2010
7,Swaps,410,84.22,,,,,april 2010 june 2010
8,Collars,228,,75.00,75.00,96.40,96.40,april 2010 june 2010
10,Swaps,414,84.22,,,,,july 2010 september 2010
11,Collars,598,,75.00,75.00,94.50|96.40,95.23,july 2010 september 2010
13,Swaps,414,84.22,,,,,october 2010 december 2010
14,Collars,598,,75.00,75.00,94.50|96.40,95.23,october 2010 december 2010


,period and type of contract,volume inmmmbtus,swapsweightedaverage,additional
2,Swaps,590,$5.86,january 2010 march 2010
4,Swaps,3163,5.85,april 2010 december 2010
6,Swaps,11863,6.36,january 2011 december 2011


,",,period and type of contract","volume inmbbls,volume inmbbls,volume inmbbls","swapsweightedaverage,swapsweightedaverage,swapsweightedaverage","collars,floors,range","collars,floors,weightedaverage","collars,ceilings,range","collars,ceilings,weightedaverage",additional
4,Swaps,905,$80.50,,,,,january 2010 june 2010
5,Collars,453,,$70.00,$70.00,$95.00,$95.00,january 2010 june 2010
7,Collars,644,,75.00,75.00,96.75,96.75,july 2010 december 2010
9,Collars,1278,,75.00,75.00,89.00,89.00,january 2011 december 2011


,period and type of contract,volume inmmmbtus,swapsweightedaverage,additional
2,Swaps,2700,$6.18,january 2010 march 2010
4,Swaps,2710,6.18,april 2010 june 2010
6,Swaps,2720,6.18,july 2010 september 2010
8,Swaps,2720,6.18,october 2010 december 2010


,period and type of contract,volume inmmmbtus,swapsweightedaverage,additional
2,Swaps,7200,$(0.62,january 2010 december 2010


In [ ]:
# %load_ext autoreload
# %autoreload 2
# import pickle
# import re
# import pandas as pd
# import numpy as np
# from IPython.display import display, HTML
# from zlu_functions import del_subset_col, set_col_headers, \
# row_header_to_col,remove_rows_for_total_value,dfs_need_transpose, remove_cols_for_fair_value,collar_fill_empty,\
# remove_empty_rows_and_cols,df_column_uniquify
# from nel_functions import delete_empty,delete_duplicated, delete_subset,get_vertically_stacked_subtables,vertically_stacked_subtable_cleaner
# from helper import volumeCleaner, dateCleaner, priceCleaner,trimDictonary,verticalSplit
# pickle_in=open('../SEC_Data/Hedge/hedge_dfs_all_new.pickle','rb')
# df_dic=pickle.load(pickle_in)

# # delete_empty(df_dic)
# df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
# del_dfs_dic_0 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
# del_dfs_dic_1 = delete_subset(df_dic)

# # # del df_dic['CPE20170228']
# df_dic_original=df_dic.copy()
# #preprocessing for all dfs
# for k in df_dic.keys():
#     for i in range(len(df_dic[k])):
#         df_dic[k][i]=set_col_headers(df_dic[k][i]) #set headers

# #Vertical Split
# for k in df_dic.keys(): #apply Sam's spliting function        
#     if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
#     elif k=='BRY20110301':
#         split_dfs = []
#         for table in df_dic[k]:
#             df1 = table.iloc[:, 0:3]
#             df2 = table.iloc[:, 3:]
#             split_dfs.append(df1)
#             split_dfs.append(df2)
#             df_dic[k]=split_dfs
#     else: df_dic[k] = verticalSplit_ZLU(df_dic[k])
#     df_dic[k]=verticalSplit_instrument(df_dic[k])
#     df_dic[k]=verticalSplit_ZLU(df_dic[k])
# #         print(k,'Original Table:',i,'-'*10)
# #         display(HTML(df.to_html()))
# #         if df.shape[1]==1: df=pd.DataFrame()
# #         elif isinstance(df, pd.DataFrame): #after new headers; ignore empty dfs
# #             df=del_subset_col(df) #delete redundant cols
# #             df=remove_rows_for_total_value(df) #if any row for total value; save it
# #             df=remove_cols_for_fair_value(df)
# #             df=row_header_to_col(df)
# #             df=dfs_need_transpose(df)
            

# #             df=collar_fill_empty(df)
# #             df_dic[k][i]=df_column_uniquify(df)
# #             df_dic[k][i]=df
# #             print(k,'Table:',i,'-'*10,'After preprocessing')
# #             display(HTML(df.to_html()))
            

# # #Vertical Split
# # for k in df_dic.keys(): #apply Sam's spliting function        
# #     if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
# #     elif k=='BRY20110301':
# #         split_dfs = []
# #         for table in df_dic[k]:
# #             df1 = table.iloc[:, 0:3]
# #             df2 = table.iloc[:, 3:]
# #             split_dfs.append(df1)
# #             split_dfs.append(df2)
# #             df_dic[k]=split_dfs
# #     else: df_dic[k] = verticalSplit_ZLU(df_dic[k])
# #     df_dic[k]=verticalSplit_instrument(df_dic[k])
# #     df_dic[k]=verticalSplit_ZLU(df_dic[k])
    
# # #Horizontally Split
# # del_dfs_dic3=get_vertically_stacked_subtables(df_dic)
# # vertically_stacked_subtable_cleaner(df_dic,del_dfs_dic3,exclusion=True, excluded_ticker=('MXC', 'OXY', 'HOC'))

# # delete_empty(df_dic)
# # df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
# # del_dfs_dic_2 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
# # del_dfs_dic_3 = delete_subset(df_dic)

# # for k in df_dic.keys():
# #     i=0
# #     for table in df_dic[k]:
# #         print(k,'Table:',i,'-'*10)
# #         display(HTML(table.to_html()))


 

In [ ]:
# %load_ext autoreload
# %autoreload 2
# import pickle
# import re
# import pandas as pd
# import numpy as np
# from IPython.display import display, HTML
# from zlu_functions import del_subset_col, set_col_headers, \
# row_header_to_col,remove_rows_for_total_value,dfs_need_transpose, remove_cols_for_fair_value,collar_fill_empty,\
# remove_empty_rows_and_cols
# from nel_functions import delete_duplicated, delete_subset,get_vertically_stacked_subtables
# from helper import volumeCleaner, dateCleaner, priceCleaner,trimDictonary#,verticalSplit
# pickle_in=open('../SEC_Data/Hedge/hedge_dfs_all_new.pickle','rb')
# df_dic=pickle.load(pickle_in)

# df_dic = trimDictonary(df_dic,[]) #remove empty list of dfs
# del_dfs_dic = delete_duplicated(df_dic) #remove duplicated dfs within a list
# del_dfs_dic2 = delete_subset(df_dic)
# del df_dic['CPE20170228']
# with open('output.csv','w') as f:
#     pass
# df_dic={'SWN20030218':df_dic['SWN20030218']}
# for k in df_dic.keys():
#     if 'SWN' in k:
#         df_lst=[]
#         for i in range(len(df_dic[k])):
#             df=df_dic[k][i]
#             display(HTML(df.to_html()))
#             print('-'*10,'Original above',k,i)
#             df=set_col_headers(df) #set headers
#             df=del_subset_col(df) #delete redundant cols
#             df=remove_rows_for_total_value(df) #if any row for total value; save it
#             df=remove_cols_for_fair_value(df)
#             df=row_header_to_col(df)
#             df=dfs_need_transpose(df)
            
#             if 'product_type' in df.index:
#                 list1=df.columns
#                 list2=df.loc[['product_type','instrument_type']].apply(lambda x: ",".join(x[:i]))
#                 df.columns=[str(a) +','+ str(b) for a,b in zip(list1,list2)]
#                 df=df.drop(['product_type','instrument_type'],axis=0)
#             display(HTML(df.to_html()))
#             df_dic[k][i]=df
#         df_dic[k]=verticalSplit_ZLU(df_dic[k])
#         df_dic[k]=verticalSplit_instrument(df_dic[k])
#         df_dic[k]=verticalSplit_ZLU(df_dic[k])
 